In [0]:
from pyspark.mllib.recommendation import ALS, Rating
from io import StringIO

In [0]:
filepath = '/FileStore/tables/hw3/ratings.dat'

In [0]:
file = sc.textFile(filepath)

In [0]:
ratings = file.map(lambda x: [int(i) for i in x.split('::')]).map(lambda x: Rating(x[0], x[1], x[2]))
ratings.take(100)

Out[37]: [Rating(user=1, product=1193, rating=5.0),
 Rating(user=1, product=661, rating=3.0),
 Rating(user=1, product=914, rating=3.0),
 Rating(user=1, product=3408, rating=4.0),
 Rating(user=1, product=2355, rating=5.0),
 Rating(user=1, product=1197, rating=3.0),
 Rating(user=1, product=1287, rating=5.0),
 Rating(user=1, product=2804, rating=5.0),
 Rating(user=1, product=594, rating=4.0),
 Rating(user=1, product=919, rating=4.0),
 Rating(user=1, product=595, rating=5.0),
 Rating(user=1, product=938, rating=4.0),
 Rating(user=1, product=2398, rating=4.0),
 Rating(user=1, product=2918, rating=4.0),
 Rating(user=1, product=1035, rating=5.0),
 Rating(user=1, product=2791, rating=4.0),
 Rating(user=1, product=2687, rating=3.0),
 Rating(user=1, product=2018, rating=4.0),
 Rating(user=1, product=3105, rating=5.0),
 Rating(user=1, product=2797, rating=4.0),
 Rating(user=1, product=2321, rating=3.0),
 Rating(user=1, product=720, rating=3.0),
 Rating(user=1, product=1270, rating=5.0),
 Rating(u

In [0]:
train, test = ratings.randomSplit([0.6, 0.4], seed=6350)

In [0]:
rank = 10
numIterations = 10
model = ALS.train(train, rank, numIterations)

In [0]:
test_data = test.map(lambda p: (p[0], p[1])) # only have userid and product
pred = model.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2])) # predict the rating
rates_and_preds = test.map(lambda r: ((r[0], r[1]), r[2])).join(pred) # join user id, product, and rating
MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.8620506014217414


In [0]:
# outputting to file

with StringIO() as file:
        
    file.write("Mean Squared Error = " + str(MSE))
        
    dbutils.fs.put("/FileStore/tables/Hw3_Q3_output.txt", file.getvalue(), True)

Wrote 39 bytes.
